In [1]:
%pylab inline

from __future__ import print_function
from __future__ import division

import pandas
import numpy
import matplotlib
import seaborn
import sklearn
import sys

from matplotlib import pyplot

Populating the interactive namespace from numpy and matplotlib


In [2]:
def index_merge(a, b):
    return a.merge(b, left_index = True, right_index = True)
pandas.DataFrame.index_merge = index_merge

# Limiting memory use

In [3]:
import resource

# Limit the memory usage to 6GByte.
resource.setrlimit(resource.RLIMIT_AS, (6e9, 6e9))

# Merging

In [47]:
subject_words = pandas.read_csv('dev/subject_word_features.csv', index_col = 'num')
body_words = pandas.read_csv('dev/body_word_features.csv', index_col = 'num')
features = features.index_merge(subject_words).index_merge(body_words)

In [ ]:
features.to_csv('dev/features.csv', index = True, header = True)

# Getting Features

In [4]:
features = pandas.read_csv('dev/features.csv', index_col = 'num').sample(frac = 1, random_state = 0)

# Testing different methods

In [5]:
from sklearn.cross_validation import train_test_split, cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier

In [6]:
X = features.drop('spam', axis = 1)
y = features.spam
%xdel features

In [57]:
word_bag = X.columns[X.columns.str.contains('_contains_')]
existence = X.columns[X.columns.str.contains('_exists')]
categorization = X.columns[X.columns.str.contains('=')]

features = ['_length', '_words', '_fields', '_avgWordLength', '_avgFieldLength']
header_features = X.columns[reduce(lambda x, y: x | y, map(X.columns.str.contains, features))]

assert (X.columns ^ (word_bag | header_features | existence | categorization)).size == 0, 'Not all features were categorized!'

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
folds = 10

In [15]:
def data(a):
    # print(u'µ = {}\nσ = {}\nmin = {}\nmax = {}\nmedian = {}'.format(a.mean(), a.std(), a.min(), a.max(), numpy.median(a)))
    print(pandas.Series(a).describe(percentiles = [.5]).drop('count'))
    return a

## Decision Tree

### As large as possible

In [10]:
# This takes WAY too long!

dt = DecisionTreeClassifier(random_state = 0)
data(cross_val_score(dt, X, y, cv = folds))

KeyboardInterrupt: 

### A little more comprehensive

In [16]:
dt = DecisionTreeClassifier(max_features = 'sqrt', max_depth = 5, random_state = 0)
data(cross_val_score(dt, X, y, cv = folds))

mean    0.973728
std     0.002218
min     0.971852
50%     0.972716
max     0.978395
dtype: float64


array([ 0.97839506,  0.97308642,  0.97234568,  0.97320988,  0.97234568,
        0.97185185,  0.97679012,  0.97222222,  0.97234568,  0.97469136])

In [19]:
sklearn.tree.export_graphviz(
    dt.tree_,
    feature_names = X.columns,
    class_names = ['ham', 'spam'],
    filled = True,
    proportion = True
)

AttributeError: 'NoneType' object has no attribute 'tree_'

## Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
rf = RandomForestClassifier(max_depth = 10, n_jobs = -1, verbose = True)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


0.98814814814814811

In [34]:
rf = RandomForestClassifier(n_estimators = 20, max_depth = 10, n_jobs = -1, verbose = True, random_state = 0)
cross_val_score(rf, X, y, n_jobs = -1, cv = 5, verbose = 1)

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.6s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.8s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.8s finished


array([ 1.        ,  1.        ,  0.98722222,  0.96635802,  0.9758642 ])

In [36]:
rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.7s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
            oob_score=False, random_state=0, verbose=True,
            warm_start=False)

In [40]:
pandas.Series(rf.feature_importances_, index = X.columns).sort_values(ascending = False)

x-cc_words                            0.059513
Return-Path_words                     0.059344
Return-Path_exists                    0.055155
x-origin_avgWordLength                0.053302
Received_avgWordLength                0.050059
Return-Path_avgWordLength             0.041361
Return-Path_avgFieldLength            0.039296
x-cc_avgWordLength                    0.032908
x-filename_words                      0.030317
from_words                            0.028809
message-id_avgWordLength              0.028523
Return-Path_fields                    0.026630
x-from_words                          0.023599
X-MS-Has-Attach_avgFieldLength        0.023237
x-bcc_length                          0.022164
body_avgWordLength                    0.021473
content-type_avgFieldLength           0.021379
Received_words                        0.021328
X-MS-Has-Attach_fields                0.021179
X-MS-TNEF-Correlator_avgWordLength    0.019977
x-cc_avgFieldLength                   0.016801
To_avgFieldLe

## Naïve Bayes

In [30]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

### Gaussian

In [26]:
nb = GaussianNB()
data(cross_val_score(nb, X, y, cv = folds))

mean    0.914012
std     0.002359
min     0.911111
50%     0.913642
max     0.917407
dtype: float64


array([ 0.91185185,  0.91358025,  0.91148148,  0.91740741,  0.91283951,
        0.91432099,  0.91111111,  0.91641975,  0.9137037 ,  0.91740741])

### Multinomial

In [31]:
nb = MultinomialNB(alpha = 1, fit_prior = False)
data(cross_val_score(nb, X, y, cv = folds))

mean    0.903457
std     0.002926
min     0.898642
50%     0.903025
max     0.908395
dtype: float64


array([ 0.90246914,  0.90061728,  0.89864198,  0.90530864,  0.9037037 ,
        0.90320988,  0.90283951,  0.90839506,  0.90209877,  0.90728395])

### Bernoulli

In [63]:
nb = BernoulliNB(alpha = 1, fit_prior = True)
data(cross_val_score(nb, X, y, cv = folds))

mean    0.951210
std     0.001046
min     0.949877
50%     0.951049
max     0.953086
dtype: float64


array([ 0.95012346,  0.95271605,  0.95148148,  0.95308642,  0.95061728,
        0.95135802,  0.95148148,  0.94987654,  0.95061728,  0.95074074])

## K Nearest Neighbours

# Whitespace

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

